In [1]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

# 初始化內存文件儲存
document_store = InMemoryDocumentStore()

/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
from haystack import Document

# 加載數據集
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

Generating train split: 100%|██████████| 151/151 [00:00<00:00, 14080.48 examples/s]


In [3]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

# 初始化文件嵌入器
doc_embedder = SentenceTransformersDocumentEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)
doc_embedder.warm_up()

/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# 創建文件嵌入並寫入文件儲存
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

Batches: 100%|██████████| 5/5 [00:03<00:00,  1.52it/s]


151

In [5]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

# 初始化文本嵌入器
text_embedder = SentenceTransformersTextEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)

In [6]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

# 初始化內存嵌入檢索器
retriever = InMemoryEmbeddingRetriever(document_store)

In [7]:
from haystack.components.builders import PromptBuilder

# 定義模板提示
template = """
根據以下信息，回答問題。
情境:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

問題: {{question}}
回答:
"""

# 初始化提示生成器
prompt_builder = PromptBuilder(template=template)

In [9]:
import os
from getpass import getpass
from haystack.components.generators import OpenAIGenerator

# 設置 OpenAI API Key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
    print("OpenAI API key set to environment variable.")
else:
    print("OpenAI API key found in environment variable.")


# 初始化 OpenAI 生成器
generator = OpenAIGenerator(model="gpt-4-turbo")

OpenAI API key found in environment variable.


In [12]:
from haystack import Pipeline

# 初始化管道
basic_rag_pipeline = Pipeline()

# 添加模組到管道
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# 連接模組
basic_rag_pipeline.connect(
    "text_embedder.embedding",
    "retriever.query_embedding"
)
basic_rag_pipeline.connect(
    "retriever",
    "prompt_builder.documents"
)
basic_rag_pipeline.connect(
    "prompt_builder",
    "llm"
)

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [16]:
# 提問
'''
Rhodes 雕像是什麼樣子的？
巴比倫花園在哪裡？
人們為什麼要建造吉薩大金字塔？
人們為什麼參觀阿耳忒彌斯神殿？
羅德島巨像的重要性是什麼？
摩索拉斯墓發生了什麼事？
羅德島巨像是怎麼崩潰的？
'''
question = "人們為什麼參觀阿耳忒彌斯神殿？"

response = basic_rag_pipeline.run({
    "text_embedder": {"text": question},
    "prompt_builder": {"question": question}
})

# 輸出答案
print(response["llm"]["replies"][0])

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.81it/s]


人們參觀阿耳忒彌斯神殿的原因多樣。首先，這座神殿是一個重要的宗教地標，供奉著女神阿耳忒彌斯，吸引了許多虔誠的朝聖者前來參拜和獻祭。其次，神殿本身的建築和藝術價值也吸引了不少遊客和學者，例如神殿的豐富細節和精緻雕刻。此外，神殿還提供了庇護，對於逃避迫害或懲罰的人來說是一個避難所，這項功能也使得許多尋求保護的人士前來。最後，許多商人、國王和觀光客也會造訪此地，他們可能是出於對神殿的好奇或者文化旅遊的目的。因此，阿耳忒彌斯神殿成為了一個結合宗教、文化和歷史的多功能景點。
